# New spaCy Pipeline

I forgot to push my final spaCy pipeline to GitHub, somehow. 🤦 So, let's make a better one.

In [1]:
%load_ext autoreload
%autoreload 2

In [38]:
x=re.compile(r'\)(?!(\s|$))', re.DEBUG)

LITERAL 41
ASSERT_NOT 1
  SUBPATTERN 1 0 0
    BRANCH
      IN
        CATEGORY CATEGORY_SPACE
    OR
      AT AT_END


In [39]:
x.sub(') ', 'dfadsfy(0)')

'dfadsfy(0)'

In [2]:
import logging
logging.basicConfig(level=logging.INFO)

In [22]:
nlp.pipe?

Signature:
nlp.pipe(
    ['texts', 'as_tuples=False', 'n_threads=2', 'batch_size=1000', 'disable=[]', 'cleanup=False', 'component_cfg=None'],
)
Docstring:
Process texts as a stream, and yield `Doc` objects in order.

texts (iterator): A sequence of texts to process.
as_tuples (bool):
    If set to True, inputs should be a sequence of
    (text, context) tuples. Output will then be a sequence of
    (doc, context) tuples. Defaults to False.
n_threads (int): Currently inactive.
batch_size (int): The number of texts to buffer.
disable (list): Names of the pipeline components to disable.
cleanup (bool): If True, unneeded strings are freed,
    to control memory use. Experimental.
component_cfg (dict): An optional dictionary with extra keyword arguments
    for specific components.
YIELDS (Doc): Documents in the order of the original text.

EXAMPLE:
    >>> texts = [u'One document.', u'...', u'Lots of documents']
    >>>     for doc in nlp.pipe(texts, batch_size=50, n_threads=4):
    >>>   

In [3]:
# Fix a spacy-Jupyter bug
# https://github.com/jupyter/notebook/issues/4369#issuecomment-459377498
get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""

In [5]:
import os
os.environ['OMP_NUM_THREADS']='1'
os.environ['MKL_NUM_THREADS']='1'

In [6]:
import gensim
import glob
import pandas as pd
import re
import spacy
import sys

sys.path.append('../../')
import tmx

INFO:summarizer.preprocessing.cleaner:'pattern' package not found; tag filters are not available for English
INFO:tmx.corpora.arxiv:spacy module loaded successfully


In [7]:
spacy.__version__

'2.1.0a11'

## The Corpus

In [5]:
!ls -l ../../data/arxiv/text/

total 56
drwxr-xr-x  3392 tshafer  staff  108544 Feb  6 13:16 0510
drwxr-xr-x  3032 tshafer  staff   97024 Feb  6 13:19 0808
drwxr-xr-x  4596 tshafer  staff  147072 Feb  6 09:16 1208
drwxr-xr-x  5713 tshafer  staff  182816 Feb  6 19:04 1403
drwxr-xr-x   183 tshafer  staff    5856 Feb  6 13:20 9210
drwxr-xr-x   846 tshafer  staff   27072 Feb  6 13:20 9604
-rw-r--r--     1 tshafer  staff   19465 Feb  7 09:11 README.md
-rw-r--r--     1 tshafer  staff    3244 Feb  7 08:17 document_counts.csv
-rw-r--r--     1 tshafer  staff     749 Feb  8 15:24 get-random-data.py


In [8]:
# This setup will let us stream the raw text via getstream()
corpus = tmx.corpora.ArxivCorpus(input=glob.glob('../../data/arxiv/text/[0-9]*'), dictionary=[])

INFO:tmx.corpora.arxiv:Input stream provided but dictionary already initialized


In [9]:
next(iter(corpus.getstream()))['text'][:50]

'Steady-state mechanical squeezing in an optomechan'

## SpaCy Tokenizer and Lemmatizer

In [158]:
nlp = spacy.load('en', disable=['ner', 'parser'])

In [247]:
# https://spacy.io/usage/linguistic-features#native-tokenizers
# https://stackoverflow.com/a/51022249/656740
def custom_tokenizer(nlp):
    def matcher(t):
        return re.match(r'\[([\w\s]+?)\s*\]\s*\1', t) #or \
#               re.match(r'([\w\s]+?)\s*\[\s*\1\s*\]', t) or \
#               re.match(r'\[[^\[\]]+\]', t) or \
#               re.match(r'\w+:\w+', t)
    return spacy.tokenizer.Tokenizer(vocab=nlp.vocab, rules=None, prefix_search=spacy.util.compile_prefix_regex(nlp.Defaults.prefixes).search, suffix_search=spacy.util.compile_suffix_regex(nlp.Defaults.suffixes).search, infix_finditer=spacy.util.compile_infix_regex(nlp.Defaults.infixes).finditer, token_match=matcher)

In [213]:
nlp.tokenizer.prefix_search('"hi')

<regex.Match object; span=(0, 1), match='"'>

In [248]:
nlp.tokenizer = custom_tokenizer(nlp)

Example:

In [249]:
doc = nlp(next(iter(corpus.getstream()))['text'])

In [250]:
[t for t in doc]

[Steady,
 -,
 state,
 mechanical,
 squeezing,
 in,
 an,
 optomechanical,
 system,
 via,
 ,
 Duffing,
 nonlinearity,
 ,
 Xin,
 -,
 You,
 Lü,
 ,
 Jie,
 -,
 Qiao,
 Liao,
 ,
 Lin,
 Tian,
 ,
 Franco,
 Nori,
 
 
 
 
 
 ,
 [,
 Introduction,
 ],
 Introduction,
 sec1[sec1]Introduction,
 [,
 sec1,
 ],
 
 ,
 Enormous,
 progress,
 has,
 been,
 achieved,
 in,
 the,
 field,
 of,
 cavity,
 optomechanics,
 ,
 in,
 the,
 past,
 few,
 years,
 .,
 Examples,
 include,
 the,
 preparation,
 ,
 of,
 mechanical,
 modes,
 to,
 their,
 quantum,
 ground,
 state,
 ,,
 the,
 demonstration,
 of,
 ,
 strong,
 optomechanical,
 coupling,
 in,
 the,
 microwave,
 and,
 optical,
 regimes,
 ,,
 and,
 ,
 the,
 coherent,
 state,
 conversion,
 between,
 cavity,
 and,
 mechanical,
 ,
 modes,
 .,
 ,
 Given,
 these,
 technological,
 advances,
 ,,
 the,
 effective,
 quantum,
 manipulation,
 ,
 of,
 mechanical,
 modes,
 becomes,
 a,
 promising,
 goal,
 .,
 
 ,
 Quantum,
 squeezing,
 of,
 mechanical,
 modes,
 is,
 one,
 of,
 the,


We need to do some scrubbing even before spaCy gets a hold of the docs:

 - remove clear LaTeX references from pandoc/detex

In [487]:
c = iter(corpus.getstream())

In [494]:
text = next(c)['text']

In [495]:
regexes = [
    # pre-reference
    r'''\[(?P<preref>[\w\s–—-]+)\s*\]\s*(?P=preref)''',
    # post-reference
    r'''(?P<postref>[\w\s–—-]+)\s*\[\s*(?P=postref)\s*\]''',
    # colon-type identifiers
    r'''\w+:\w+''',
    # refNum
    r'''[a-zA-Z]+[0-9]+''',
    # email addresses
    r'''[^\s]+@[^\s]+'''
]

In [17]:
os.cpu_count()

INFO:tmx.corpora.arxiv:spacy module loaded successfully


8

In [20]:
len(os.sched_get_priority_max())

TypeError: Required argument 'policy' (pos 1) not found

In [496]:
'|'.join(regexes)

'\\[(?P<preref>[\\w\\s–—-]+)\\s*\\]\\s*(?P=preref)|(?P<postref>[\\w\\s–—-]+)\\s*\\[\\s*(?P=postref)\\s*\\]|\\w+:\\w+|[a-zA-Z]+[0-9]+|[^\\s]+@[^\\s]+'

In [497]:
#text = re.sub('|'.join([r'(%s)' % r for r in regexes]), '', re.sub(r'(?<=\w)\n(?=\w)', ' ', text))

In [498]:
doc = nlp(text)

In [16]:
nlp.pipe?

INFO:tmx.corpora.arxiv:spacy module loaded successfully


Signature:
nlp.pipe(
    ['texts', 'as_tuples=False', 'n_threads=2', 'batch_size=1000', 'disable=[]', 'cleanup=False', 'component_cfg=None'],
)
Docstring:
Process texts as a stream, and yield `Doc` objects in order.

texts (iterator): A sequence of texts to process.
as_tuples (bool):
    If set to True, inputs should be a sequence of
    (text, context) tuples. Output will then be a sequence of
    (doc, context) tuples. Defaults to False.
n_threads (int): Currently inactive.
batch_size (int): The number of texts to buffer.
disable (list): Names of the pipeline components to disable.
cleanup (bool): If True, unneeded strings are freed,
    to control memory use. Experimental.
component_cfg (dict): An optional dictionary with extra keyword arguments
    for specific components.
YIELDS (Doc): Documents in the order of the original text.

EXAMPLE:
    >>> texts = [u'One document.', u'...', u'Lots of documents']
    >>>     for doc in nlp.pipe(texts, batch_size=50, n_threads=4):
    >>>   

In [480]:
[
    (token.text, token.lemma_)
    for token in doc
    if re.search(r'\w', token.text)
    and token.lemma_ != '-PRON-'
][-100:]

[('1983', '1983'),
 ('G.', 'g.'),
 ('Aarts', 'aarts'),
 ('F.', 'f.'),
 ('A.', 'a.'),
 ('James', 'james'),
 ('E.', 'e.'),
 ('Seiler', 'seiler'),
 ('and', 'and'),
 ('I.', 'i.'),
 ('-O.', '-o.'),
 ('Stamatescu', 'stamatescu'),
 ('Phys', 'phys'),
 ('Lett', 'lett'),
 ('B', 'b'),
 ('687', '687'),
 ('2010', '2010'),
 ('.0617', '.0617'),
 ('hep', 'hep'),
 ('lat', 'lat'),
 ('J.', 'j.'),
 ('Ambjorn', 'ambjorn'),
 ('M.', 'm.'),
 ('Flensburg', 'flensburg'),
 ('and', 'and'),
 ('C.', 'c.'),
 ('Peterson', 'peterson'),
 ('Nucl', 'nucl'),
 ('Phys', 'phys'),
 ('B', 'b'),
 ('275', '275'),
 ('1986', '1986'),
 ('375', '375'),
 ('A.', 'a.'),
 ('Mollgaard', 'mollgaard'),
 ('and', 'and'),
 ('K.', 'k.'),
 ('Splittorff', 'splittorff'),
 ('Phys', 'phys'),
 ('Rev', 'rev'),
 ('D', 'd'),
 ('88', '88'),
 ('2013', '2013'),
 ('116007', '116007'),
 ('.4335', '.4335'),
 ('hep', 'hep'),
 ('lat', 'lat'),
 ('E.', 'e.'),
 ('Seiler', 'seiler'),
 ('D.', 'd.'),
 ('Sexty', 'sexty'),
 ('and', 'and'),
 ('I.', 'i.'),
 ('-O.', '-o.

## SpaCy with futures

In [9]:
!python -m spacy download en

Looking in indexes: http://nexus.cho.elderresearch.com:8081/repository/eri-pypi-all/simple/, https://pypi.python.org/simple
    100% |████████████████████████████████| 11.1MB 15.9MB/s 
  Found existing installation: en-core-web-sm 2.0.0
    Uninstalling en-core-web-sm-2.0.0:
      Successfully uninstalled en-core-web-sm-2.0.0
  Running setup.py install for en-core-web-sm ... done
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/miniconda3/envs/Belair-TM/lib/python3.6/site-packages/en_core_web_sm
-->
/usr/local/miniconda3/envs/Belair-TM/lib/python3.6/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [10]:
nlp = spacy.load('en', disable=['ner', 'parser'])

In [11]:
def test_function(doc):
    return [t.lemma_ for t in nlp_obj(doc)]

In [13]:
from concurrent import futures

In [14]:
nlp_obj = spacy.load('en')
with futures.ProcessPoolExecutor() as ppe:
    jobs = {}
    for i in range(10000):
        jobs[ppe.submit(test_function, 'here is a fun document, yo')] = i

KeyboardInterrupt: 

In [7]:
nlp(next(iter(corpus.getstream())))

Signature:
nlp.pipe(
    ['texts', 'as_tuples=False', 'n_threads=2', 'batch_size=1000', 'disable=[]', 'cleanup=False', 'component_cfg=None'],
)
Docstring:
Process texts as a stream, and yield `Doc` objects in order.

texts (iterator): A sequence of texts to process.
as_tuples (bool):
    If set to True, inputs should be a sequence of
    (text, context) tuples. Output will then be a sequence of
    (doc, context) tuples. Defaults to False.
n_threads (int): Currently inactive.
batch_size (int): The number of texts to buffer.
disable (list): Names of the pipeline components to disable.
cleanup (bool): If True, unneeded strings are freed,
    to control memory use. Experimental.
component_cfg (dict): An optional dictionary with extra keyword arguments
    for specific components.
YIELDS (Doc): Documents in the order of the original text.

EXAMPLE:
    >>> texts = [u'One document.', u'...', u'Lots of documents']
    >>>     for doc in nlp.pipe(texts, batch_size=50, n_threads=4):
    >>>   

In [12]:
text = next(iter(corpus.getstream()))['text']

In [14]:
_ = list(nlp.pipe([text for i in range(20)]))

INFO:tmx.corpora.arxiv:spacy module loaded successfully


In [15]:
_ = list(nlp(text) for i in range(20))